### Code to activate the Javascript for video
---


In [39]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  """
  The canvas will display the img_data 
  at the same time capture an image from the webcam
  """
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

### Drawing Functions
---

In [40]:
import numpy as np
from PIL import Image
import base64
import io

def js_reply_to_image(js_reply):
    """
    Usage: 
          To extract the image from the JS object returned from take_photo
    input: 
          js_reply: JavaScript object, contain image from webcam
    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    return image_array

def drawing_array_to_bytes(drawing_array):
    """
    Usage: 
          To convert an image to image bytes. 
          Image bytes is used by JS for display.
    input: 
          drawing_array: image RGB size 512 x 512 
    output: 
          drawing_bytes: string, encoded from drawing_array
    """
    drawing_PIL = Image.fromarray(drawing_array, 'RGB')
    iobuf = io.BytesIO()
    drawing_PIL.save(iobuf, format='png')
    drawing_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))
    return drawing_bytes

---
### Mount to Google Drive
Mount to your Google Drive to access the images and model files.

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Image processing function
---
Replace the codes in this section to process the live images based on your requirement.

In [42]:
# !pip install dlib
# !pip install face_recognition

In [43]:
import face_recognition
import pandas as pd
import cv2

# Load the stored face encodings
face_encodings_csv = "/content/drive/MyDrive/ADLCV/dataset/Face/KnownFaceEncodings_3.csv"
Known_db = pd.read_csv(face_encodings_csv)
known_face_encodings = Known_db.iloc[:, 1:].to_numpy()
known_face_name = Known_db.iloc[:, 0].to_numpy()

def process_image(image):
  """
  Usage: 
          To process the input image.
    input: 
          image: image RGB size 512 x 512 
    output: 
          processed_image: processed image RGB size 512 x 512 
  """
  image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=2)
  unknown_face_encodings = face_recognition.face_encodings(image, known_face_locations=face_locations)

  # There might be more than one person in the photo, so we need to loop over each face we found
  for unknown_face_encoding, face_location in zip(unknown_face_encodings, face_locations):

      # Test the distance between each unknown face with all known faces
      FaceDistance = face_recognition.face_distance(known_face_encodings, unknown_face_encoding)
      # Test of the unknown face is found in all known faces
      results = face_recognition.compare_faces(known_face_encodings, unknown_face_encoding, tolerance=0.52)
      
      # Obtain the index with the shortest distance
      minFaceDistance = np.argmin(FaceDistance)
      # Calculate the confidence level
      confidence = (1 - FaceDistance[minFaceDistance])*100

      # Obtain the name from the found index
      if results[minFaceDistance]:
        name = known_face_name[minFaceDistance]
      else:
        name = "Unknown"
      
      # Extract the corrdinates of the face location
      y1, x2, y2, x1 = face_location

      cv2.rectangle(image_bgr, (x1,y1), (x2,y2), (0,0,255))
      cv2.putText(image_bgr, name, (x1,y1), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255))
      
  processed_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

  return(processed_image)

## Main program
---

In [44]:
start_input()
label_html = 'Capturing...'
img_data = ''
count = 0 
while True:
    js_reply = take_photo(label_html, img_data)
    if not js_reply:
        break
        
    image = js_reply_to_image(js_reply)
    processed_image = process_image(image) 
    img_data = drawing_array_to_bytes(processed_image)


<IPython.core.display.Javascript object>